# Notebook 1: Meshes

Introducing meshes and how to visualise them



In [1]:
#|  echo: false

# This is required to fix pyvista 
# (visualisation) crashes in interactive notebooks (including on binder)

import nest_asyncio
nest_asyncio.apply()

In [2]:
#| output: false # Suppress warnings in html version

import underworld3 as uw
import numpy as np
import sympy

[esdhcp-137.anu.edu.au:32788] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.esdhcp-137.501/jf.0/3384344576/sm_segment.esdhcp-137.501.c9b90000.0 could be created.


In [3]:
mesh = uw.meshing.uw.meshing.CubedSphere(
    radiusOuter=1.0,
    radiusInner=0.547,
    numElements=8,
    simplex=False,
    verbose=False,
)

There are other pre-built meshes you can try. This is a cuboid divided into regular tetrahedra:

```python
mesh_usb = uw.meshing.UnstructuredSimplexBox(
    minCoords = (-1.0, -1.0, -1.0),
    maxCoords = (+1.0, +1.0, +1.0),
    cellSize = 0.2,
    regular=True,
    verbose=False,
)
```

and this is a two-dimensional annulus mesh


```python
mesh_ann = uw.meshing.Annulus(
    radiusOuter=1.0,
    radiusInner=0.547,
    cellSize= 0.5,
    cellSizeOuter=0.033,
    cellSizeInner=0.05,
    verbose=False,
)
```

The meshing infrastructure for underworld3 is documented here: https://underworldcode.github.io/underworld3/main_api/underworld3/meshing.html

In [ ]:


import pyvista as pv
import underworld3.visualisation as vis

# Try out each one !

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["z"] = vis.scalar_fn_to_pv_points(pvmesh, mesh.CoordinateSystem.X[2])

if mesh.dim==3:
    pvmesh_c = pvmesh.clip( normal='z', crinkle=True, inplace=False, origin=(0.0,0.0,0.01))

pl = pv.Plotter(window_size=(750, 750))
pl.add_mesh(pvmesh_c, show_edges=True, show_scalar_bar=False, opacity=1.0)
pl.add_mesh(pvmesh, show_edges=True, show_scalar_bar=False, opacity=0.3)


# Save and show the mesh
pl.export_html("html5/spherical_mesh_plot.html")

In [6]:
#| fig-cap: "Interactive Image: Spherical shell mesh cut in half and overlain with transparent view of the whole mesh. Cubed sphere discretisation using hexahedral elements"

from IPython.display import IFrame
IFrame(src="html5/spherical_mesh_plot.html", width=600, height=400)

### Coordinate systems

The mesh has an associated "natural" coordinate system (usually Cartesian), but it may also have other, more convenient, coordinate systems. 

For example, the spherical mesh above has a Cartesian coordinate system which is the one used to navigate the mesh and describe the location of each point. It also has a spherical $(r, \theta, \phi)$ system which is **symbolic** and can be expanded in terms of the Cartesian coordinates.

In [5]:
## The coordinate system

display(mesh.CoordinateSystem.X)
display(mesh.CoordinateSystem.R)
display(uw.function.expression.sub_all(mesh.CoordinateSystem.R))

Matrix([[N.x, N.y, N.z]])

Matrix([[{ {  } r }, { { \,\! } \theta }, { { \,\!\,\! } \phi }]])

Matrix([[sqrt(N.x**2 + N.y**2 + N.z**2), acos(N.z/sqrt(N.x**2 + N.y**2 + N.z**2)), atan2(N.y, N.x)]])